In [718]:
import pennylane as qml
import random
import numpy as np
# Define the number of qubits
n_qubits = 3
shots = 500

# Initialize a device
dev = qml.device('default.qubit', wires=n_qubits, shots=shots)

# Define the initial state preparation function
def initial_state():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])

# Define the weak error function
def apply_weak_error():
    # Randomly select one of the three qubits
    qubit_to_flip = random.randint(0, 2)
    # Apply a small rotation around the X axis to simulate a weak bit-flip error
    epsilon = np.pi / 100
    qml.RX(epsilon, wires=qubit_to_flip)

@qml.qnode(dev)
def circuit():
    initial_state()
    apply_weak_error()
    
    # Measure the stabilizers
    Z1Z2 = qml.sample(qml.PauliZ(0) @ qml.PauliZ(1))
    Z1Z3 = qml.sample(qml.PauliZ(0) @ qml.PauliZ(2))
    
    # Define error correction based on the stabilizer measurements
    if Z1Z2 == 1 and Z1Z3 == 1:
        pass
    elif Z1Z2 == -1 and Z1Z3 == -1:
        qml.PauliX(wires=0)
    elif Z1Z2 == -1 and Z1Z3 == 1:
        qml.PauliX(wires=1)
    elif Z1Z3 == 1 and Z1Z2 == -1:
        qml.PauliX(wires=2)
    
    # Measure the corrected state after error correction
    corrected_state = qml.counts(wires=[0, 1, 2])
    
    # Return stabilizer measurements and corrected state
    return Z1Z2, Z1Z3, corrected_state

# Run the circuit
Z1Z2, Z1Z3, corrected_state = circuit()

# Print stabilizer measurements and corrected state
print("Corrected state:", corrected_state)





Corrected state: {'000': 269, '111': 231}
